<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/pandas/pandas_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Clean Up Pandas Data

Here we should how to clean up Pandas data, in particular what do with about missing data, as well as what to do with invalid data.

Consider a survey.  If you send people a survey you cannot control in all cases what questions they will answer.  And you cannot anticipate what data they might put that is invalid.  For example, they might leave off some data.  Or they might enter a number in a question that is only supposed to be text.

So here we show you how to :

* generate some random data that is purposefully not clean
* drop duplicate rows
* get rid of rows that have missing values
* convert missing values to something else, like a fixed value of the average of all the other values in the colummn
* delete outliers, which are obvious typos.  For example, here we enter some salaries as 1 million while everyone else is around 100,000.  So that's most likely a mistake (since these are employees and not company owners or the CEO).
* apply a custom function to every row to do whatever special checking you want
* check for missing values
* show different ways to check for numbers or strings and how to convert those when they are of the wrong type

# Bad Data

Below we create some data and purposely add some bad data.  What do we do with this data?  Do we fix it? We we erase it?  Do we drop entire rows?  If this was a survey of 100 question you would not want to delete the rows because almost every person will either put bad data or not answer all questions.

In this data we have:

1. blank data in numeric columns
2. NaN (not a number) data in numeric columns
3. blank values in text columns
4. numbers that are multiple standard deviation away from the other answers, suggesting a type
5. Number in a Yes-No column

We will show how to clean up each.




In [253]:
import numpy as np
import pandas as pd

import random

def makedata():
  mean=10000
  std=25

  # this code creates random data.  It adds invalid and missing values to give us data to work with.

  cols = [("name", str), ("education", str),
     ("age", np.int8), ("city",str), ("id", np.int8), ("email", str), ("salary", np.int8),
        ("citizen", ["Y", "N"])]

  words = [np.NaN, "", "abc", "def", "ghi", "jkl", "mno", "pqr"]

  records = []

  for i in range(20):

    data = {}

    for c in cols:

      if c[1] == np.int8:
        if random.randint(0,5)==5:
            data[c[0]] = np.NaN
        else:
            data[c[0]] = abs(int(random.gauss(mean, std)))

      if c[0] == "citizen":
        if random.randint(0,5)==5:
            data[c[0]] = random.randint(0,10)
        else:
            data[c[0]] =c[1][random.randint(0,1)]

      if c[1] == str and c[0] != "citizen":
        data[c[0]] = words[random.randint(0,len(words)-1)]

      if (c[0] == "salary") & (random.randint(0,5)==0):
            data[c[0]] = 1000000

      if (c[0] == "age"):
          data[c[0]] = random.randint(20,25)

    records.append(data)

  df=pd.DataFrame(records)

  return df

df = makedata()
df


,name,education,age,city,id,email,salary,citizen
0,abc,NaN,24,pqr,9978.0,abc,9957.0,Y
1,jkl,pqr,23,jkl,10041.0,ghi,1000000.0,N
2,abc,ghi,21,pqr,10021.0,,9982.0,N
3,,NaN,23,def,9969.0,ghi,10011.0,Y
4,,mno,20,pqr,NaN,mno,NaN,N
5,jkl,ghi,20,ghi,NaN,abc,9997.0,Y
6,abc,ghi,20,mno,9997.0,mno,10075.0,N
7,def,pqr,22,jkl,10021.0,ghi,1000000.0,Y
8,abc,,21,NaN,9988.0,ghi,NaN,9
9,def,ghi,23,pqr,10019.0,mno,9974.0,2


In [254]:
# check if column in series is empty

df['name'].isnull()


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17     True
18    False
19    False
Name: name, dtype: bool

In [255]:
# Here we check if

(df.isna() | df.isnull())


,name,education,age,city,id,email,salary,citizen
0,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False
4,False,False,False,False,True,False,True,False
5,False,False,False,False,True,False,False,False
6,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False
8,False,False,False,True,False,False,True,False
9,False,False,False,False,False,False,False,False


In [256]:
np.NaN == None

False

In [257]:
df['salary']

0        9957.0
1     1000000.0
2        9982.0
3       10011.0
4           NaN
5        9997.0
6       10075.0
7     1000000.0
8           NaN
9        9974.0
10       9978.0
11       9992.0
12       9984.0
13       9994.0
14    1000000.0
15      10028.0
16      10014.0
17      10011.0
18       9988.0
19       9987.0
Name: salary, dtype: float64

In [258]:
# draw from mean()
# we can use the mean() function on a series.  A series is one column.
# Then we can replace blank values with the mean.  This is logical.



In [259]:
df['salary'].mean()

174998.44444444444

In [260]:
df['citizen']

0     Y
1     N
2     N
3     Y
4     N
5     Y
6     N
7     Y
8     9
9     2
10    Y
11    N
12    Y
13    N
14    Y
15    Y
16    N
17    6
18    N
19    3
Name: citizen, dtype: object

In [261]:
# Yes, No columns cannot contain numbers
# NaN is OK as we can get rid of it in a second step plus certain functions like mean()
# will ignore it.  coerce means convert to NaN on error

df['citizen'] = df['citizen'].apply(lambda x: x if x in ['Y', 'N'] else "")

df['citizen']

0     Y
1     N
2     N
3     Y
4     N
5     Y
6     N
7     Y
8      
9      
10    Y
11    N
12    Y
13    N
14    Y
15    Y
16    N
17     
18    N
19     
Name: citizen, dtype: object

In [262]:
df['salary']

0        9957.0
1     1000000.0
2        9982.0
3       10011.0
4           NaN
5        9997.0
6       10075.0
7     1000000.0
8           NaN
9        9974.0
10       9978.0
11       9992.0
12       9984.0
13       9994.0
14    1000000.0
15      10028.0
16      10014.0
17      10011.0
18       9988.0
19       9987.0
Name: salary, dtype: float64

In [263]:
# drop outliers, too many standard deviations away
# if the income - mean > 2 std the replace then drop the row
# remeber to use inplace=True


mean = df['salary'].mean()
std = df['salary'].std()

# Define threshold for outliers (e.g., values more than 2 standard deviations away from the mean)
threshold = 2

# calculate
# find with .loc
# drop
# inplace=True


df.drop(df.loc[abs(df['salary'] - mean) > 2 * std].index, inplace=True)

df



,name,education,age,city,id,email,salary,citizen
0,abc,NaN,24,pqr,9978.0,abc,9957.0,Y
2,abc,ghi,21,pqr,10021.0,,9982.0,N
3,,NaN,23,def,9969.0,ghi,10011.0,Y
4,,mno,20,pqr,NaN,mno,NaN,N
5,jkl,ghi,20,ghi,NaN,abc,9997.0,Y
6,abc,ghi,20,mno,9997.0,mno,10075.0,N
8,abc,,21,NaN,9988.0,ghi,NaN,
9,def,ghi,23,pqr,10019.0,mno,9974.0,
10,jkl,NaN,23,mno,9947.0,,9978.0,Y
11,,NaN,24,ghi,10027.0,NaN,9992.0,N


In [264]:
abs(1000000 - mean) > 2 * std

True

In [265]:
mean

174998.44444444444

In [266]:
std

379648.3861505696

In [267]:
sMean=df['salary'].mean()

df['salary'] = df['salary'].fillna(sMean)

df

,name,education,age,city,id,email,salary,citizen
0,abc,NaN,24,pqr,9978.0,abc,9957.000000,Y
2,abc,ghi,21,pqr,10021.0,,9982.000000,N
3,,NaN,23,def,9969.0,ghi,10011.000000,Y
4,,mno,20,pqr,NaN,mno,9998.133333,N
5,jkl,ghi,20,ghi,NaN,abc,9997.000000,Y
6,abc,ghi,20,mno,9997.0,mno,10075.000000,N
8,abc,,21,NaN,9988.0,ghi,9998.133333,
9,def,ghi,23,pqr,10019.0,mno,9974.000000,
10,jkl,NaN,23,mno,9947.0,,9978.000000,Y
11,,NaN,24,ghi,10027.0,NaN,9992.000000,N


In [268]:
# drop all numbers in Yes/No answer

df['citizen'] = df['citizen'].apply(lambda x: x if isinstance(x,str) else np.NaN)
df['citizen']


0     Y
2     N
3     Y
4     N
5     Y
6     N
8      
9      
10    Y
11    N
12    Y
13    N
15    Y
16    N
17     
18    N
19     
Name: citizen, dtype: object

In [269]:
# Here is how to drop all rows that have any NaN values in any column.
# we will leave off inplace=True so that we don't delete all the data that we need for this lesson.
# remember that if you don't put inplace=True and you have not assigned the df.somefunction() to
# some value then you have effectively done nothing as nothing has changed

df.dropna(how='any')
df

,name,education,age,city,id,email,salary,citizen
0,abc,NaN,24,pqr,9978.0,abc,9957.000000,Y
2,abc,ghi,21,pqr,10021.0,,9982.000000,N
3,,NaN,23,def,9969.0,ghi,10011.000000,Y
4,,mno,20,pqr,NaN,mno,9998.133333,N
5,jkl,ghi,20,ghi,NaN,abc,9997.000000,Y
6,abc,ghi,20,mno,9997.0,mno,10075.000000,N
8,abc,,21,NaN,9988.0,ghi,9998.133333,
9,def,ghi,23,pqr,10019.0,mno,9974.000000,
10,jkl,NaN,23,mno,9947.0,,9978.000000,Y
11,,NaN,24,ghi,10027.0,NaN,9992.000000,N


In [270]:
# here run a function on a Pandas series, which is a single column.  But since we are
# sending a single row-column combination we can work with it as we would with any
# Python primitive (meaning built-in type.  Pandas and Numpy are extension of Python into new types.)
# In the next example we show how to work with an entire row, where we have all columns we can work with


def toUpper(s):
    if isinstance(s,str):
      return s.upper()


df['city']=df['city'].apply(toUpper)
df

,name,education,age,city,id,email,salary,citizen
0,abc,NaN,24,PQR,9978.0,abc,9957.000000,Y
2,abc,ghi,21,PQR,10021.0,,9982.000000,N
3,,NaN,23,DEF,9969.0,ghi,10011.000000,Y
4,,mno,20,PQR,NaN,mno,9998.133333,N
5,jkl,ghi,20,GHI,NaN,abc,9997.000000,Y
6,abc,ghi,20,MNO,9997.0,mno,10075.000000,N
8,abc,,21,None,9988.0,ghi,9998.133333,
9,def,ghi,23,PQR,10019.0,mno,9974.000000,
10,jkl,NaN,23,MNO,9947.0,,9978.000000,Y
11,,NaN,24,GHI,10027.0,NaN,9992.000000,N


In [271]:
# axis = 1 means row
# axis = 0 means column

# the important point to note there is we send the entire row in as a paramters. thus all the columns
# are available to use.  remember to send back the entire row after you have updated any of the columns.

def wholeRow(row):
  if isinstance(row['city'],str):
     row['city'] = row['city'].lower()
     return row


df=df.apply(wholeRow, axis=1)
df

,name,education,age,city,id,email,salary,citizen
0,abc,NaN,24.0,pqr,9978.0,abc,9957.000000,Y
2,abc,ghi,21.0,pqr,10021.0,,9982.000000,N
3,,NaN,23.0,def,9969.0,ghi,10011.000000,Y
4,,mno,20.0,pqr,NaN,mno,9998.133333,N
5,jkl,ghi,20.0,ghi,NaN,abc,9997.000000,Y
6,abc,ghi,20.0,mno,9997.0,mno,10075.000000,N
8,None,None,NaN,None,NaN,None,NaN,None
9,def,ghi,23.0,pqr,10019.0,mno,9974.000000,
10,jkl,NaN,23.0,mno,9947.0,,9978.000000,Y
11,,NaN,24.0,ghi,10027.0,NaN,9992.000000,N


# Eliminate Duplicates

Here we show how to get rid of duplicate rows.  


In [272]:
df = pd.DataFrame({
    "a" : [1,2,3,3,3,3,5,6,7,7,7,7]
})

df.groupby('a')['a'].count()


a
1    1
2    1
3    4
5    1
6    1
7    4
Name: a, dtype: int64

In [273]:
df.value_counts()

a
3    4
7    4
1    1
2    1
5    1
6    1
Name: count, dtype: int64

In [274]:
df.drop_duplicates(inplace=True)
df.groupby('a')['a'].count()

a
1    1
2    1
3    1
5    1
6    1
7    1
Name: a, dtype: int64

In [275]:
df['a'].value_counts()

a
1    1
2    1
3    1
5    1
6    1
7    1
Name: count, dtype: int64

In [276]:
df = makedata()

df.groupby('age')['age'].count()

age
20    3
21    1
22    3
23    4
24    3
25    6
Name: age, dtype: int64

In [277]:
# keep='first' Mark duplicates as True except for the first occurrence

df['duplicate'] = df.duplicated(subset=['age'], keep='first')

df.sort_values("age")

,name,education,age,city,id,email,salary,citizen,duplicate
1,mno,abc,20,def,9976.0,ghi,10033.0,N,False
12,mno,ghi,20,ghi,9997.0,,10013.0,N,True
5,abc,,20,mno,10018.0,mno,10013.0,Y,True
6,mno,,21,,10017.0,pqr,10007.0,5,False
9,def,jkl,22,NaN,9899.0,,9999.0,Y,False
15,jkl,def,22,abc,9982.0,NaN,1000000.0,Y,True
19,,abc,22,NaN,10063.0,mno,NaN,N,True
7,,jkl,23,,10001.0,ghi,1000000.0,N,True
11,ghi,pqr,23,,9976.0,pqr,9980.0,N,True
3,NaN,abc,23,mno,10001.0,NaN,1000000.0,Y,True


In [278]:
# drop those where duplicate is true.  Notice the gap in index values show which rows were dropped

df=df[df['duplicate'] == False]

In [279]:
df.sort_values("age")

,name,education,age,city,id,email,salary,citizen,duplicate
1,mno,abc,20,def,9976.0,ghi,10033.0,N,False
6,mno,,21,,10017.0,pqr,10007.0,5,False
9,def,jkl,22,NaN,9899.0,,9999.0,Y,False
0,jkl,NaN,23,mno,9993.0,pqr,9995.0,7,False
13,,pqr,24,mno,9969.0,jkl,10032.0,Y,False
2,ghi,jkl,25,abc,NaN,mno,1000000.0,N,False
